In [59]:
import cv2
import numpy as np

# 學號
student_id = "M11202130"

# 定義裁切函數
def crop_and_paste_image(original_image, transformed_image, reference_pts):
    # 將點轉換為numpy陣列
    pts = np.array(reference_pts, dtype=np.int32)

    # 創建一個黑色背景的遮罩
    mask = np.zeros(original_image.shape[:2], dtype=np.uint8)

    # 在遮罩上繪製白色四邊形
    cv2.fillPoly(mask, [pts], 255)
    
    # 將遮罩應用到裁剪後的圖像上，只保留四點內的範圍
    cropped_transformed_image = cv2.bitwise_and(transformed_image, transformed_image, mask=mask)

    # 在原始圖片上貼上裁剪後的圖片
    result_image = original_image.copy()
    result_image[mask == 255] = cropped_transformed_image[mask == 255]

    return result_image

# 初始化全局變量
clicked_points = []

# 定義滑鼠回調函數
def mouse_callback(event, x, y, flags, param):
    global clicked_points
    if event == cv2.EVENT_LBUTTONDOWN:
        # 決定標記顏色 前四點為紅 之後為綠
        color = (0, 0, 255) if len(clicked_points) < 4 else (0, 255, 0)

        # 在原始圖像上標記一個點
        cv2.drawMarker(resized_image, (x, y), color, cv2.MARKER_CROSS, 20, 2)  # 紅色十字
        cv2.imshow('image', resized_image)

        # 將點座標轉換為原始圖片大小的座標並儲存
        original_x = int(x / resize_factor)
        original_y = int(y / resize_factor)
        clicked_points.append((original_x, original_y))

        if len(clicked_points) == 8:  # 當累積了8個點時
            cv2.setMouseCallback('image', lambda *args: None)  # 停止監聽滑鼠事件
            calculate_homography()

# 計算 Homography
def calculate_homography():
    # 前四個點是選取範圍，後四個點是參考座標
    image1_pts  = np.array(clicked_points[:4], dtype=np.float32)
    image2_pts = np.array(clicked_points[4:], dtype=np.float32)

    # 計算 兩個 Homography
    H1, _ = cv2.findHomography(image1_pts , image2_pts)
    H2, _ = cv2.findHomography(image2_pts, image1_pts )

    # 將 Homography 應用於原始大小的圖像
    transformed_image_1 = cv2.warpPerspective(original_image, H1, (original_image.shape[1], original_image.shape[0]))
    transformed_image_2 = cv2.warpPerspective(original_image, H2, (original_image.shape[1], original_image.shape[0]))

    # 裁剪圖像
    result_image = crop_and_paste_image(original_image, transformed_image_1, image2_pts)
    result_image = crop_and_paste_image(result_image, transformed_image_2, image1_pts )

    # 在結果圖片左下角加上學號
    cv2.putText(result_image, student_id, (10, result_image.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # 顯示交換後的圖像
    cv2.imshow('Result Image', result_image)
    cv2.imwrite(f'{student_id}.jpg', result_image)  # 將結果保存為 學號.jpg
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 讀取原始圖片
original_image = cv2.imread('Swap_ArtGallery.jpg')

# 縮小原始圖片
resize_factor = 0.5
resized_image = cv2.resize(original_image, (int(original_image.shape[1] * resize_factor), int(original_image.shape[0] * resize_factor)))

# 顯示縮小後的圖片並設置滑鼠回調函數
cv2.imshow('image', resized_image)
cv2.setMouseCallback('image', mouse_callback)

# 等待按鍵
cv2.waitKey(0)
cv2.destroyAllWindows()
